# Getting started with InstaNovo

<a target="_blank" href="https://colab.research.google.com/github/instadeepai/InstaNovo/blob/main/notebooks/getting_started_with_instanovo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<!-- <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/instadeepai/InstaNovo/blob/main/notebooks/getting_started_with_instanovo.ipynb">
<img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/> </a> -->

In this notebook, we demo InstaNovo, a transformer neural network with the ability to translate fragment ion peaks into the sequence of amino acids that make up the studied peptide(s). We evaluate the model on the yeast test fold of nine-species dataset.

![](https://raw.githubusercontent.com/instadeepai/InstaNovo/main/docs/assets/graphical_abstract.jpeg)

**Links:**

- Nature Machine Intelligence Paper: [**InstaNovo enables diffusion-powered de novo peptide sequencing in large-scale proteomics experiments**](https://www.nature.com/articles/s42256-025-01019-5) \
  Kevin Eloff, Konstantinos Kalogeropoulos, Amandla Mabona, Oliver Morell, Rachel Catzel, Esperanza Rivera-de-Torre, Jakob Berg Jespersen, Wesley Williams, Sam P. B. van Beljouw, Marcin J. Skwark, Andreas Hougaard Laustsen, Stan J. J. Brouns, Anne Ljungars, Erwin M. Schoof, Jeroen Van Goey, Ulrich auf dem Keller, Karim Beguir, Nicolas Lopez Carranza, Timothy P. Jenkins
- Code: [GitHub](https://github.com/instadeepai/InstaNovo)

**Important:**

It is highly recommended to run this notebook in an environment with access to a GPU. If you are running this notebook in Google Colab:

- In the menu, go to `Runtime > Change Runtime Type > T4 GPU`

## Loading the InstaNovo model

We first install the latest instanovo from PyPi

In [ ]:
import os
import sys
if 'google.colab' in sys.modules or 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    # Suppress TensorFlow warnings
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    # Install torchvision when running on Google Colab to prevent errors
    !uv pip install --system "instanovo[cu124]>=1.1.1" pyopenms-viz torchvision tf-nightly
else:
    !uv pip install "instanovo[cu124]>=1.1.1" pyopenms-viz

In [ ]:
# Filter warnings and set logging level
import warnings
import logging
warnings.filterwarnings("ignore", module="matplotlib")
warnings.filterwarnings("ignore", module="torch")
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("rdkit").setLevel(logging.WARNING)

We can use `instanovo version` to check the version of InstaNovo (the transformer-based model), InstaNovo+ (the diffusion-based model) and some of their dependencies.

In [ ]:
!instanovo version

Import the transformer-based InstaNovo model.

In [1]:
from instanovo.transformer.model import InstaNovo

[04/08/25 18:26:51] INFO     Enabling RDKit 2024.09.6 jupyter extensions                             ]8;id=920697;file://h:\Github\InstaNovo\.venv\Lib\site-packages\rdkit\__init__.py\__init__.py]8;;\:]8;id=596823;file://h:\Github\InstaNovo\.venv\Lib\site-packages\rdkit\__init__.py#33\33]8;;\

Set the device to GPU if available (recommended), otherwise use CPU.

In [6]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

InstaNovo supports automatic model downloads. You can see the IDs of the pretrained models that are available.

In [2]:
InstaNovo.get_pretrained()

['instanovo-v1.1.0', 'instanovo-v1.0.0']

And download the model checkpoint given the ID.

In [5]:
model, config = InstaNovo.from_pretrained("instanovo-v1.1.0")
model = model.to(device).eval()

[04/08/25 18:33:01] INFO     Model instanovo-v1.1.0 already cached at                                  ]8;id=886329;file://H:\Github\InstaNovo\instanovo\transformer\model.py\model.py]8;;\:]8;id=200785;file://H:\Github\InstaNovo\instanovo\transformer\model.py#242\242]8;;\
                             C:\Users\bujiw\.cache\instanovo\instanovo-v1.1.0.ckpt                                 

NameError: name 'device' is not defined

Alternatively, you can also download the model checkpoint directly from the [InstaNovo releases](https://github.com/instadeepai/InstaNovo/releases) page.

## Loading the nine-species dataset
Download the [yeast test fold of the nine-species dataset](https://huggingface.co/datasets/InstaDeepAI/instanovo_ninespecies_exclude_yeast) dataset from HuggingFace.

We can use our SpectrumDataFrame to download this directly. SpectrumDataFrame is a special data class used by InstaNovo to read and write from multiple formats, including mgf, mzml, mzxml, pandas, polars, HuggingFace, etc.

In [12]:
from instanovo.utils import SpectrumDataFrame

sdf = SpectrumDataFrame.from_huggingface(
    "InstaDeepAI/ms_ninespecies_benchmark",
    is_annotated=True,
    shuffle=False,
    split="test[:10%]",  # Let's only use a subset of the test data for faster inference in this notebook
)

In [13]:
sdf.to_pandas().head(5)

,sequence,modified_sequence,precursor_mz,precursor_charge,mz_array,intensity_array
0,IVSWYDNEYGYSTR,IVSWYDNEYGYSTR,876.8974,2,"[101.2592, 102.09755, 115.0502, 115.23178, 119...","[844.33905, 823.5147, 4108.16, 902.05914, 863...."
1,TFFVGGNFK,TFFVGGNFK,508.7639,2,"[101.40765, 103.484726, 105.461, 114.2153, 120...","[661.61536, 625.5021, 726.2794, 795.70575, 101..."
2,ISVQDIDIK,ISVQDIDIK,515.7919,2,"[103.03921, 117.05487, 119.081215, 129.10231, ...","[1857.5409, 6994.2676, 1061.3785, 1567.5293, 9..."
3,TTPSFVGFTDTER,TTPSFVGFTDTER,729.3486,2,"[102.78241, 107.25752, 119.63341, 128.69043, 1...","[730.7964, 680.5609, 837.06836, 1129.5052, 254..."
4,SPIIIQTSNGGAAYFAGK,SPIIIQTSNGGAAYFAGK,897.9721,2,"[136.06091, 138.11862, 147.11348, 157.09729, 1...","[2217.4023, 795.4153, 889.5029, 1443.5614, 192..."


Let's quickly plot the spectrum in the first row

In [ ]:
import pandas as pd

pd.options.plotting.backend = "ms_matplotlib"
row = sdf[0]
row_df = pd.DataFrame({"mz": row["mz_array"], "intensity": row["intensity_array"]})
row_df.plot(
    kind="spectrum",
    x="mz",
    y="intensity",
    annotate_mz=True,
    bin_method="none",
    annotate_top_n_peaks=5,
    aggregate_duplicates=True,
    title=f"Mass spectrum of {row['sequence']}",
);

In [ ]:
from instanovo.transformer.dataset import SpectrumDataset, collate_batch

ds = SpectrumDataset(
    sdf,
    model.residue_set,
    config.get("n_peaks", 200),
    return_str=True,
    annotated=True,
)

In [ ]:
from torch.utils.data import DataLoader

# When using SpectrumDataFrame, workers and shuffle is handled internally.
dl = DataLoader(ds, batch_size=64, shuffle=False, num_workers=0, collate_fn=collate_batch)

In [ ]:
batch = next(iter(dl))

spectra, precursors, spectra_mask, peptides, _ = batch
spectra = spectra.to(device)
precursors = precursors.to(device)

## Decoding

We have three options for decoding:
- Greedy Search
- Beam Search
- Knapsack Beam Search

For the best results and highest peptide recall, use **Knapsack Beam Search**. 
For fastest results (over 10x speedup), use **Greedy Search**.

We generally use a beam size of 5 for Beam Search and Knapsack Beam Search, a higher beam size should increase recall at the cost of performance and vice versa.

_Note: in our findings, greedy search has similar performance as knapsack beam search at 5% FDR. I.e. if you plan to filter at 5% FDR anyway, use greedy search for optimal performance._

### Greedy Search and Beam Search

Greedy search is used when `num_beams=1`, and beam search is used when `num_beams>1`

In [ ]:
from instanovo.inference import BeamSearchDecoder, GreedyDecoder

num_beams = 1  # Change this, defaults are 1 or 5

if num_beams > 1:
    decoder = BeamSearchDecoder(model=model)
else:
    decoder = GreedyDecoder(model=model)

### Knapsack Beam Search

Setup knapsack beam search decoder. This may take a few minutes.

In [ ]:
from pathlib import Path
from instanovo.constants import MASS_SCALE
from instanovo.inference.knapsack import Knapsack
from instanovo.inference.knapsack_beam_search import KnapsackBeamSearchDecoder

num_beams = 5

def _setup_knapsack(model: InstaNovo) -> Knapsack:
    # Cannot allow negative masses in knapsack graph
    if "(-17.03)" in model.residue_set.residue_masses:
        model.residue_set.residue_masses["(-17.03)"] = 1e3
    if "[UNIMOD:385]" in model.residue_set.residue_masses:
        model.residue_set.residue_masses["[UNIMOD:385]"] = 1e3

    residue_masses = dict(model.residue_set.residue_masses.copy())
    if any(x < 0 for x in residue_masses.values()):
        raise NotImplementedError(
            "Negative mass found in residues, this will break the knapsack graph. "
            "Either disable knapsack or use strictly positive masses"
        )
    for special_residue in list(model.residue_set.residue_to_index.keys())[:3]:
        residue_masses[special_residue] = 0
    residue_indices = model.residue_set.residue_to_index
    return Knapsack.construct_knapsack(
        residue_masses=residue_masses,
        residue_indices=residue_indices,
        max_mass=4000.00,
        mass_scale=MASS_SCALE,
    )


knapsack_path = Path("./checkpoints/knapsack/")

if not knapsack_path.exists():
    print("Knapsack path missing or not specified, generating...")
    knapsack = _setup_knapsack(model)
    decoder = KnapsackBeamSearchDecoder(model, knapsack)
    print(f"Saving knapsack to {knapsack_path}")
    knapsack_path.parent.mkdir(parents=True, exist_ok=True)
    knapsack.save(knapsack_path)
else:
    print("Knapsack path found. Loading...")
    decoder = KnapsackBeamSearchDecoder.from_file(model=model, path=knapsack_path)

## Inference time 🚀

Evaluating a single batch...

In [ ]:
from instanovo.inference import ScoredSequence

with torch.no_grad():
    p = decoder.decode(
        spectra=spectra,
        precursors=precursors,
        beam_size=num_beams,
        max_length=config["max_length"],
    )

preds = [x.sequence if isinstance(x, ScoredSequence) else [] for x in p]
probs = [x.sequence_log_probability if isinstance(x, ScoredSequence) else -float("inf") for x in p]

### Confidence probabilities
The model returns per-residue confidences in the form of token log-probabilities. We can visualize these or use them as part of a workflow.

In [ ]:
from typing import Optional

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from instanovo.inference.beam_search import ScoredSequence


def plot_residue_confidence(prediction: ScoredSequence, peptide: Optional[str] = None) -> None:
    if not prediction:
        return
    ticks = list(range(len(prediction.sequence)))
    token_probabilities = np.exp(prediction.token_log_probabilities[:len(ticks)])
    sequence_confidence = np.exp(prediction.sequence_log_probability)

    _, ax = plt.subplots()
    bars = sns.barplot(x=ticks, y=token_probabilities, errorbar=None, ax=ax)

    # Increase Y-axis limit to create space for text labels
    ax.set_ylim(0, max(token_probabilities) * 1.2)

    # Add numbers above bars with a slanted angle
    for bar, prob in zip(bars.patches, token_probabilities):
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            float(height) + 0.02,
            f"{float(prob):.4f}",
            ha="center",
            va="bottom",
            fontsize=9,
            color="black",
            rotation=45,
        )

    # Check if any residue contains a PTM (e.g., "S(+79.97)")
    has_ptm = any("(" in res and ")" in res for res in prediction.sequence)

    # Set labels
    x_label = f"    Prediction: {''.join(prediction.sequence)}"
    if peptide is not None:
        x_label += f"\nGround truth: {peptide}"
    ax.set_xlabel(x_label)
    ax.set_ylabel("Confidence Probability")

    # Set title with sequence confidence
    ax.set_title(
        f"Residue Confidence per Position\nSequence Probability: {sequence_confidence:.4f}"
    )

    # Set X-ticks
    ax.set_xticks(ticks)
    ax.set_xticklabels(
        prediction.sequence,
        rotation=45 if has_ptm else 0,
        ha="right" if has_ptm else "center",
    )

    plt.show()

For a spectrum that is sequenced correctly, the sequence probability and per-residue probabilities are uniformly high:

In [ ]:
plot_residue_confidence(p[-1], peptides[-1])

For another spectrum which is sequenced incorrectly, the sequence probability is low and the per-residue probabilities of the incorrectly sequenced residues (up to isomerism) are lower than of those correctly sequenced:

In [ ]:
plot_residue_confidence(p[0], peptides[0])

These examples suggest the model is fairly well calibrated.

### Evaluation

In [ ]:
from instanovo.utils.metrics import Metrics

metrics = Metrics(model.residue_set, config["isotope_error_range"])

We include a residue remapping to ensure our input dataset can be mapped to the format the model vocabulary expects.

In [ ]:
model.residue_set.update_remapping({
  "M(ox)": "M[UNIMOD:35]",
  "M(+15.99)": "M[UNIMOD:35]",
  "S(p)": "S[UNIMOD:21]", # Phosphorylation
  "T(p)": "T[UNIMOD:21]",
  "Y(p)": "Y[UNIMOD:21]",
  "S(+79.97)": "S[UNIMOD:21]",
  "T(+79.97)": "T[UNIMOD:21]",
  "Y(+79.97)": "Y[UNIMOD:21]",
  "Q(+0.98)": "Q[UNIMOD:7]", # Deamidation
  "N(+0.98)": "N[UNIMOD:7]",
  "Q(+.98)": "Q[UNIMOD:7]",
  "N(+.98)": "N[UNIMOD:7]",
  "C(+57.02)": "C[UNIMOD:4]", # Carboxyamidomethylation
  "(+42.01)": "[UNIMOD:1]", # Acetylation
  "(+43.01)": "[UNIMOD:5]", # Carbamylation
  "(-17.03)": "[UNIMOD:385]"})

In [ ]:
aa_precision, aa_recall, peptide_recall, peptide_precision = metrics.compute_precision_recall(
    peptides, preds
)
peptide_recall

Evaluating on the yeast test fold of the nine-species dataset:

In [ ]:
from tqdm.notebook import tqdm

preds = []
targs = []
probs = []

for _, batch in tqdm(enumerate(dl), total=len(dl)):
    spectra, precursors, _, peptides, _ = batch
    spectra = spectra.to(device)
    precursors = precursors.to(device)

    with torch.no_grad():
        p = decoder.decode(
            spectra=spectra,
            precursors=precursors,
            beam_size=config["n_beams"],
            max_length=config["max_length"],
        )

    preds += [x.sequence if isinstance(x, ScoredSequence) else [] for x in p]
    probs += [
        x.sequence_log_probability if isinstance(x, ScoredSequence) else -float("inf") for x in p
    ]
    targs += list(peptides)

### Evaluation metrics

Model performance without filtering:

In [ ]:
aa_precision, aa_recall, peptide_recall, peptide_precision = metrics.compute_precision_recall(
    targs, preds
)
aa_error_rate = metrics.compute_aa_er(targs, preds)
auc = metrics.calc_auc(targs, preds, np.exp(pd.Series(probs)))

print(f"amino acid error rate:    {aa_error_rate:.5f}")
print(f"amino acid precision:     {aa_precision:.5f}")
print(f"amino acid recall:        {aa_recall:.5f}")
print(f"peptide precision:        {peptide_precision:.5f}")
print(f"peptide recall:           {peptide_recall:.5f}")
print(f"area under the PR curve:  {auc:.5f}")

### We can find a threshold to ensure a desired FDR:

Model performance at 5% FDR:

In [ ]:
fdr = 5 / 100  # Desired FDR

_, threshold = metrics.find_recall_at_fdr(targs, preds, np.exp(probs), fdr=fdr)
aa_precision_fdr, aa_recall_fdr, peptide_recall_fdr, peptide_precision_fdr = (
    metrics.compute_precision_recall(targs, preds, np.exp(probs), threshold=threshold)
)
print(f"Performance at {fdr*100:.1f}% FDR:\n")
print(f"amino acid precision:     {aa_precision_fdr:.5f}")
print(f"amino acid recall:        {aa_recall_fdr:.5f}")
print(f"peptide precision:        {peptide_precision_fdr:.5f}")
print(f"peptide recall:           {peptide_recall_fdr:.5f}")
print(f"area under the PR curve:  {auc:.5f}")
print(f"confidence threshold:     {threshold:.5f}  <-- Use this as a confidence cutoff")

_Note: to reproduce the results of the paper, the entire Yeast test set should be evaluated with the 0.1.7 release of InstaNovo._

### Saving the predictions...

In [ ]:
pred_df = pd.DataFrame(
    {
        "targets": targs,
        "tokenized_predictions": preds,
        "predictions": ["".join(x) for x in preds],
        "log_probabilities": probs,
    }
)
pred_df.head()

In [ ]:
pred_df.to_csv("predictions_knapsack_beam_search.csv", index=False)

## InstaNovo+: Iterative Refinement with a Diffusion Model

In this section, we show how to refine the predictions from the transformer model with a diffusion model.

First, we download the model checkpoint.

In [3]:
from instanovo.diffusion.multinomial_diffusion import InstaNovoPlus

InstaNovoPlus.get_pretrained()

['instanovoplus-v1.1.0-alpha']

In [7]:
diffusion_model, diffusion_config = InstaNovoPlus.from_pretrained("instanovoplus-v1.1.0-alpha")
diffusion_model = diffusion_model.to(device).eval()

[04/08/25 18:33:10] INFO     Model instanovoplus-v1.1.0-alpha already cached at        ]8;id=297134;file://H:\Github\InstaNovo\instanovo\diffusion\multinomial_diffusion.py\multinomial_diffusion.py]8;;\:]8;id=339437;file://H:\Github\InstaNovo\instanovo\diffusion\multinomial_diffusion.py#354\354]8;;\
                             C:\Users\bujiw\.cache\instanovo\instanovoplus-v1.1.0-alph                             
                             a.ckpt                                                                                

                    INFO     Loading model instanovoplus-v1.1.0-alpha (remote)         ]8;id=481411;file://H:\Github\InstaNovo\instanovo\diffusion\multinomial_diffusion.py\multinomial_diffusion.py]8;;\:]8;id=715546;file://H:\Github\InstaNovo\instanovo\diffusion\multinomial_diffusion.py#357\357]8;;\

[04/08/25 18:33:11] INFO     Loading InstaNovoPlus model to device: cpu.               ]8;id=302281;file://H:\Github\InstaNovo\instanovo\diffusion\multinomial_diffusion.py\multinomial_diffusion.py]8;;\:]8;id=457178;file://H:\Github\InstaNovo\instanovo\diffusion\multinomial_diffusion.py#259\259]8;;\

In [9]:
for k in diffusion_config.keys():
    print(k, diffusion_config.get(k))

tb_summarywriter s3://dtu-denovo-s-2e6da747d6d34f62-outputs/output/ba9e4dc3-5bb4-4b96-b3ec-5be42af8fe66/tensorboard/
seed 101
warmup_iters 50000
learning_rate 5e-05
weight_decay 0
train_batch_size 128
n_gpu 1
gradient_clip_val 5
predict_batch_size 128
fp16 True
compile_model True
epochs 30
num_sanity_val_steps 10
console_logging_steps 2000
tensorboard_logging_steps 500
report_to neptune
run_name instanovo_extended_massivekb
train_subset 1
valid_subset 0.01
val_check_interval 0.2
lazy_loading True
max_shard_size 1000000
preshuffle_shards False
perform_data_checks True
validate_precursor_mass False
verbose_loading True
save_model True
model_save_folder_path checkpoints/instanovoplus-extended-massivekb
ckpt_interval 50000
train_from_scratch True
resume_checkpoint None
blacklist /mnt/instanovo-data-kyber/identity_splits_parquet/blacklist.csv
device cuda
time_steps 200
vocab_size 33
layers 12
dim 768
nheads 8
dropout 0.1
attention_type wavlm
wavlm_num_bucket 140
wavlm_max_dist 280
dim_feedf

In [71]:
list(diffusion_config.get('residues').keys())

['G',
 'A',
 'S',
 'P',
 'V',
 'T',
 'C',
 'L',
 'I',
 'N',
 'D',
 'Q',
 'K',
 'E',
 'M',
 'H',
 'F',
 'R',
 'Y',
 'W',
 'M[UNIMOD:35]',
 'C[UNIMOD:4]',
 'N[UNIMOD:7]',
 'Q[UNIMOD:7]',
 'S[UNIMOD:21]',
 'T[UNIMOD:21]',
 'Y[UNIMOD:21]',
 '[UNIMOD:1]',
 '[UNIMOD:5]',
 '[UNIMOD:385]',
 '[PAD]',
 '[EOS]',
 '[SOS]']

In [56]:
from instanovo.utils import SpectrumDataFrame

sdf = SpectrumDataFrame.from_huggingface(
    "InstaDeepAI/ms_ninespecies_benchmark",
    is_annotated=True,
    shuffle=False,
    split="test[:100%]",  # Let's only use a subset of the test data for faster inference in this notebook
)

In [113]:
import re
from collections import Counter
import pandas as pd

# Define vocab (amino acids and modifications)
vocab1 = {
  "M(ox)": "M[UNIMOD:35]",
  "M(+15.99)": "M[UNIMOD:35]",
  "S(p)": "S[UNIMOD:21]", # Phosphorylation
  "T(p)": "T[UNIMOD:21]",
  "Y(p)": "Y[UNIMOD:21]",
  "S(+79.97)": "S[UNIMOD:21]",
  "T(+79.97)": "T[UNIMOD:21]",
  "Y(+79.97)": "Y[UNIMOD:21]",
  "Q(+0.98)": "Q[UNIMOD:7]", # Deamidation
  "N(+0.98)": "N[UNIMOD:7]",
  "Q(+.98)": "Q[UNIMOD:7]",
  "N(+.98)": "N[UNIMOD:7]",
  "C(+57.02)": "C[UNIMOD:4]", # Carboxyamidomethylation
  "(+42.01)": "[UNIMOD:1]", # Acetylation
  "(+43.01)": "[UNIMOD:5]", # Carbamylation
  "(-17.03)": "[UNIMOD:385]"
}

# Define vocab2 (standard amino acids and special tokens)
vocab2 = {
    0: "[PAD]",
    1: "[SOS]",
    2: "[EOS]",
    3: "G",
    4: "A",
    5: "S",
    6: "P",
    7: "V",
    8: "T",
    9: "C",
    10: "L",
    11: "I",
    12: "N",
    13: "D",
    14: "Q",
    15: "K",
    16: "E",
    17: "M",
    18: "H",
    19: "F",
    20: "R",
    21: "Y",
    22: "W",
}

# Combine vocab and vocab2
vocab = set(vocab2.values()) | set(vocab1.keys())

# Reverse vocab for token lookup (to map from amino acid to index)
aa_to_idx = {aa: idx for idx, aa in enumerate(vocab)}

# Define valid amino acids (including modified amino acids)
valid_aas = vocab 

# Regex pattern to match standard AAs and modified ones like M(+15.99)
aa_pattern = re.compile(r"[A-Z](?:\(\+\d+(?:\.\d+)?\))?")

# Initialize Counter to track amino acid frequencies (across the entire sequence)
aa_counts = Counter()

# Process each sequence (here, I assume you already have the sequences in `sdf`)
sequences = sdf.to_pandas()['sequence']

# Process sequences and count amino acids
for seq in sequences:
    # Extract amino acids using regex pattern

    aa_seq = [aa for aa in aa_pattern.findall(seq) if aa in valid_aas]
    aa_seq = aa_seq[:39]  # max 39 tokens to allow EOS at end
    aa_seq.append("[EOS]")

    # Pad to length 40 if needed
    while len(aa_seq) < 40:
        aa_seq.append("[PAD]")
    # Count occurrences of each amino acid in the sequence (ignoring positions)
    aa_counts.update(aa_seq)

# Ensure all valid amino acids are included, even those with count 0
full_counts = {aa: aa_counts.get(aa, 0) for aa in valid_aas}

# Convert the full counts to a DataFrame
aa_counts_df = pd.DataFrame(full_counts.items(), columns=["Amino Acid", "Count"])

aa_counts_df['Converted Amino Acid'] = aa_counts_df['Amino Acid'].map(vocab1).fillna(aa_counts_df['Amino Acid'])
aa_counts_df['Converted Amino Acid'] = pd.Categorical(aa_counts_df['Converted Amino Acid'], 
                                                      categories=list(diffusion_config.get('residues').keys()), 
                                                      ordered=True)
aa_counts_df.drop(columns = ['Amino Acid'], inplace= True)

# Optionally, sort the amino acids by count
# aa_counts_df = aa_counts_df.sort_values(by="Count", ascending=False).reset_index(drop=True)

# aa_counts_df
# Display the result
aa_counts_df = aa_counts_df.groupby('Converted Amino Acid').sum().reset_index()
# # aa_counts_df
# aa_counts_df.set_index('Converted Amino Acid', inplace=True)
aa_counts_df


vocab = {0: '[PAD]', 1: '[SOS]', 2: '[EOS]', 3: 'G', 4: 'A', 5: 'S', 6: 'P', 7: 'V', 8: 'T', 9: 'C',
            10: 'L', 11: 'I', 12: 'N', 13: 'D', 14: 'Q', 15: 'K', 16: 'E', 17: 'M', 18: 'H', 19: 'F', 20: 'R',
            21: 'Y', 22: 'W', 23: 'M[UNIMOD:35]', 24: 'C[UNIMOD:4]', 25: 'N[UNIMOD:7]', 26: 'Q[UNIMOD:7]', 27:
            'S[UNIMOD:21]', 28: 'T[UNIMOD:21]', 29: 'Y[UNIMOD:21]', 30: '[UNIMOD:1]', 31: '[UNIMOD:5]', 32:
            '[UNIMOD:385]'}
    
distributions = np.zeros(len(vocab))
for idx, aa_name in vocab.items():
    # Check if the amino acid is present in the 'Converted Amino Acid' column of aa_counts_df
    if aa_name in aa_counts_df['Converted Amino Acid'].values:
        # Get the count for this amino acid
        count = aa_counts_df[aa_counts_df['Converted Amino Acid'] == aa_name]['Count'].values[0]
        # Assign the count to the appropriate index in the distributions array
        distributions[idx] = count

distributions += 1e-6
distributions/=distributions.sum()
distributions.shape

C:\Users\bujiw\AppData\Local\Temp\ipykernel_17656\2709441885.py:101: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  aa_counts_df = aa_counts_df.groupby('Converted Amino Acid').sum().reset_index()


(33,)

In [103]:
from collections import Counter

# Example order_list

# Count occurrences of each element
counter = Counter(order_list)

# Find elements that appear more than once
duplicates = [item for item, count in counter.items() if count > 1]

print(duplicates)


['[PAD]', '[SOS]', '[EOS]']


In [38]:
for i in sdf.to_pandas()['sequence'].unique():
    print(i)

IVSWYDNEYGYSTR
TFFVGGNFK
ISVQDIDIK
TTPSFVGFTDTER
SPIIIQTSNGGAAYFAGK
EAIDFFAR
NQWFFSK
IWIPVNR
IDKSQVDEIVIVGGSTR
VIGIDGGEGKEEIFR
IVSTIDANFADKYDEVK
VHGEEDPTKK
ISENGQR
IQEEERER
VAQVQK
DAHGVTQAR
KGNNTANATNSANTVQK
ITPKPEQK
VAGKVQPEDNK
YAGEVSHDDK
ETEKANADANEDNNVDEK
NGKPNSTTSSIK
SRGESDDSINR
GSIDEQHPR
ITPKPEEK
IEQFEIK
HFHTHKPAR
GTETEESINKR
QTHPDTGISQK
MASQDKDGKTTDADESEKHNYQEQYNK
NYDPQR
EVMQR
ITAATNAKQ
VISSDAR
SKQEASQM(+15.99)AAMAEK
ASEAIKPDSQK
IPPDQQR
AQNPMR
ITSSQVR
DISTNQR
AGVDVDEKGQGKNEETSGEGGEDKNEPSSK
ENSVKDGEEEEDEEDEDEK
ASASARSR
VSNEETSETIKK
NSDGDTKDEGDNKDEDDDEDDDDDDDDEDDDDEAPTK
IQQGMR
QSENSVKDGEEEEDEEDEDEK
GESDDSINR
YMGAAK
THGAPTDEVR
HVGDM(+15.99)GNVKTDEN(+.98)GVAK
KVDADADADC(+57.02)DANGDSNGRVDC(+57.02)K
YGATSTNPAK
AAEAGETGAATSATEGDNNNNTAAGDKK
HVGDM(+15.99)GNVKTDENGVAK
SSYGSSSNDDSYGSSNNDDSYGSSNKKK
SM(+15.99)VEEAEASGR
SSYGSNNDDSYGSNNDDSYGSSNKKK
NQGDYNPQTGR
THSGPTTASNPAPSSTNSSSAPSATNSKQER
NGKKIEDAEGQENAASSE
DITTNQR
SVYDSR
MSREDDNEENQGDDENEENVDSQR
TAEQVAAER
FGQAAK
SKGENVKEPESEKETAENNDSDSEADTS

Next we create a decoder object.

In [10]:
from instanovo.inference.diffusion import DiffusionDecoder

diffusion_decoder = DiffusionDecoder(model=diffusion_model)

Then we prepare the inference data loader using predictions from the InstaNovo transformer model.

In [11]:
import polars as pl

diffusion_df = sdf.to_polars(return_lazy=False)

diffusion_df = diffusion_df.with_columns(pl.Series("sequence", pred_df["tokenized_predictions"]))

diffusion_df = diffusion_df.with_columns(pl.Series("original_peptide", targs)) # preserve targets

diffusion_df = diffusion_df.filter(pl.col("sequence") != []) # remove rows with empty predictions
targs = diffusion_df["original_peptide"].to_list() # update targets accordingly

diffusion_sdf = SpectrumDataFrame.from_polars(df=diffusion_df, is_annotated=False, shuffle=False)

diffusion_ds = SpectrumDataset(
    diffusion_sdf,
    diffusion_model.residues,
    diffusion_model.config.get("n_peaks", 200),
    return_str=False,
    annotated=True,
    peptide_pad_length=diffusion_model.config.get("max_length", 30),
    diffusion=True,
    reverse_peptide=False,  # we do not reverse peptide for diffusion
    tokenize_peptide=False,
)

diffusion_dl = DataLoader(
    diffusion_ds,
    batch_size=64,
    num_workers=0,  # sdf requirement, handled internally
    shuffle=False,  # sdf requirement, handled internally
    collate_fn=collate_batch,
)

NameError: name 'sdf' is not defined

Finally, we predict sequences by iterating over the spectra and refining the InstaNovo predictions.

In [ ]:
predictions = []
log_probs = []

for batch in tqdm(diffusion_dl, total=len(diffusion_dl)):
    spectra, precursors, spectra_padding_mask, peptides, _ = batch
    spectra = spectra.to(device)
    precursors = precursors.to(device)
    spectra_padding_mask = spectra_padding_mask.to(device)
    peptides = peptides.to(device)
    with torch.no_grad():
        batch_predictions, batch_log_probs = diffusion_decoder.decode(
            spectra=spectra,
            spectra_padding_mask=spectra_padding_mask,
            precursors=precursors,
            initial_sequence=peptides,
        )
    predictions.extend(batch_predictions)
    log_probs.extend(batch_log_probs)

Iterative refinement improves performance on this sample of the Nine Species dataset. (To replicate the performance reported in the paper, you would need to evaluate on the entire dataset.) 

In [ ]:
(
    aa_precision_refined,
    aa_recall_refined,
    peptide_recall_refined,
    peptide_precision_refined,
) = metrics.compute_precision_recall(targs, predictions=predictions)
aa_error_rate_refined = metrics.compute_aa_er(targs, predictions)
auc_refined = metrics.calc_auc(targs, predictions, np.exp(pd.Series(log_probs)))

print(f"amino acid error rate:     {aa_error_rate_refined:.5f}")
print(f"amino acid precision:      {aa_precision_refined:.5f}")
print(f"amino acid recall:         {aa_recall_refined:.5f}")
print(f"peptide precision:         {peptide_precision_refined:.5f}")
print(f"peptide recall:            {peptide_recall_refined:.5f}")
print(f"area under the ROC curve:  {auc_refined:.5f}")

In [ ]:
print(f"Decrease in AA error rate:     {100 * (aa_error_rate - aa_error_rate_refined):.2f}%")
print(f"Increase in AA precision:      {100 * (aa_precision_refined - aa_precision):.2f}%")
print(f"Increase in AA recall:         {100 * (aa_recall_refined - aa_recall):.2f}%")
print(f"Increase in peptide precision: {100 * (peptide_precision_refined - peptide_precision):.2f}%")
print(f"Increase in peptide recall:    {100 * (peptide_recall_refined - peptide_recall):.2f}%")
print(f"Increase in AUC:               {100 * (auc_refined - auc):.2f}%")

In [ ]:
diffusion_predictions = pd.DataFrame(
    {
        "targets": targs,
        "tokenized_predictions": predictions,
        "predictions": ["".join(x) for x in predictions],
        "log_probabilities": log_probs,
    }
)
diffusion_predictions.head()

In [ ]:
diffusion_predictions.to_csv("diffusion_predictions.csv", index=False)